### Lectura de archivos y librerias

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Esta función sirve para visualizar todo el dataframe dentro de jupyter, pues por default se corta
from IPython.display import HTML

def view(df: pd.DataFrame):
    
    return HTML(df.head(10).to_html())

El archivo 'TCA_iar_Reservaciones.csv' ya tiene las columnas renombradas en base al archivo 'Matriz de Trazabilidad y tablas fuente.xlsx' (realizado por el archivo 'file_transformation.ipynb'). Estos nombres son más intuitivos que los que están presentes en la base de datos por defecto.

In [ ]:
df = pd.read_csv("../Data_Sources/TCA_iar_Reservaciones.csv")
iar_agencias = pd.read_csv("../Data_Sources/TCA_iar_Agencias.csv")
iar_canales = pd.read_csv("../Data_Sources/TCA_iar_canales.csv")
iar_estatus_reservaciones = pd.read_csv("../Data_Sources/TCA_iar_estatus_reservaciones.csv")
iar_segmentos_comp = pd.read_csv("../Data_Sources/TCA_iar_Segmentos_Comp.csv")
iar_tipos_habitacion = pd.read_csv("../Data_Sources/TCA_iar_Tipos_Habitaciones.csv")

C:\Users\diego\AppData\Local\Temp\ipykernel_18468\3965315338.py:1: DtypeWarning: Columns (22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data_sources/TCA_iar_Reservaciones.csv")


In [6]:
# df.drop([column for column in df.columns if column.startswith('aa_')])
columns = df.columns
columns_to_drop = []

for column in columns:
    
    if column.startswith("aa_"):
        columns_to_drop.append(column)
        
df.drop(columns = columns_to_drop, inplace = True)

del columns, columns_to_drop

In [7]:
tablas_iar = {
    'reservacioens': df,
    'agencias': iar_agencias,
    'canales': iar_canales,
    'estatus_reservaciones': iar_estatus_reservaciones,
    'segmentos_comp': iar_segmentos_comp,
    'tipos_habitacion': iar_tipos_habitacion
}

### Limpieza de nombres de columnas

Reemplazamos carácteres especiales y hacer todas las letras minúsculas

In [8]:
def clean_columns(x: pd.Index) -> pd.Index:
    
    x = (pd.Series(x).apply(lambda y: '_'.join(y.split()))
                     .str.lower()
                     .str.replace('á', 'a')
                     .str.replace('é', 'e')
                     .str.replace('í', 'i')
                     .str.replace('ó', 'o')
                     .str.replace('ú', 'u')
                     .str.replace('ñ', 'n')
                     .str.replace('º', '')
                     .str.replace('°', '')
                     .str.replace('(', '')
                     .str.replace(')', '')
                     .str.replace('-', '')
        )
    
    return pd.Index(x)

In [9]:
for nombre, tabla in tablas_iar.items():
    tabla.columns = clean_columns(tabla.columns)

del tablas_iar

In [10]:
# Agencias
iar_agencias = iar_agencias[['id_agencia', 'cliente']]
iar_agencias = iar_agencias.rename(columns = {'cliente': 'agencia'})
df = pd.merge(df, iar_agencias, on = 'id_agencia', how = 'left')
df.drop('id_agencia', axis = 1, inplace = True)

# Canales
iar_canales = iar_canales[['id_canal', 'canal']]
df = pd.merge(df, iar_canales, on = 'id_canal', how = 'left')
df.drop('id_canal', axis = 1, inplace = True)

# Estatus reservaciones
iar_estatus_reservaciones = iar_estatus_reservaciones[['id_estatus_reservaciones', 'estatus_de_la_reservacion']]
df = pd.merge(df, iar_estatus_reservaciones, on = 'id_estatus_reservaciones', how = 'left')
df.drop('id_estatus_reservaciones', axis = 1, inplace = True)

# Segmentos complementarios
iar_segmentos_comp = iar_segmentos_comp[['id_segmento_comp', 'segmento_alterno']]
df = pd.merge(df, iar_segmentos_comp, on = 'id_segmento_comp', how = 'left')
df.drop('id_segmento_comp', axis = 1, inplace = True)

# Tipos habitacion
iar_tipos_habitacion = iar_tipos_habitacion[['id_tipo_habitacion', 'tipo_habitacion_nombre']]
df = pd.merge(df, iar_tipos_habitacion, on = 'id_tipo_habitacion', how = 'left')
df.drop('id_tipo_habitacion', axis = 1, inplace = True)

del iar_agencias, iar_canales, iar_estatus_reservaciones, iar_tipos_habitacion, iar_segmentos_comp#, iar_hotedo, iar_paises_origen, 

### Eliminar columnas vacias

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203002 entries, 0 to 203001
Data columns (total 39 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id_reserva                   203002 non-null  int64  
 1   fecha_hoy                    203002 non-null  object 
 2   h_res_fec                    203002 non-null  int64  
 3   h_res_fec_ok                 203002 non-null  object 
 4   h_res_fec_okt                203002 non-null  object 
 5   h_num_per                    203002 non-null  int64  
 6   h_num_adu                    203002 non-null  int64  
 7   h_num_men                    203002 non-null  int64  
 8   cuartos_noche                203002 non-null  int64  
 9   habitaciones                 203002 non-null  int64  
 10  id_programa                  203002 non-null  int64  
 11  id_paquete                   203002 non-null  int64  
 12  id_empresa                   203002 non-null  int64  
 13 

Se observan dos columnas que son 100% valores nulos, es decir, están vacías.

In [11]:
df.dropna(how = 'all', axis = 1, inplace = True)
df.isna().sum().sum()

0

### Revision de registros duplicados y su eliminacion

In [13]:
duplicated_data = df.drop('id_reserva', axis = 1).duplicated()
duplicated_data = duplicated_data[duplicated_data]
# duplicated_data.sum()

In [14]:
df.drop(duplicated_data.index, inplace = True)

### Seleccion de columnas de fecha y sus comprobaciones

In [15]:
def format_date_columns(df, date_columns):
    
    for column in date_columns:
        df[column] = df[column].astype(str).str.strip()
        df[column] = pd.to_datetime(df[column], errors = 'coerce').dt.strftime('%Y%m%d')
    
    return df

In [16]:
date_columns = ['fecha_hoy',
                'h_res_fec',
                'h_res_fec_ok',
                'h_res_fec_okt',
                'fecha_llegada',
                'h_fec_lld_ok',
                'h_fec_lld_okt',
                'h_fec_reg',
                'h_fec_reg_ok',
                'h_fec_reg_okt',
                'fecha_salida',
                'fecha_confirmacion_dormidas',
                'fecha',
                'h_ult_cam_fec',
                'fecha_confirmacion_pickup',
                'h_ult_cam_fec_okt']

df = format_date_columns(df = df, date_columns = date_columns)

In [17]:
def date_comparison_check(df_subset: pd.DataFrame):
    
    return df_subset[~df_subset.nunique(axis = 1).eq(1)].index.tolist()

In [18]:
column_check_reservacion = date_columns[0:4]
mismatched_fechas_reservacion = date_comparison_check(df[column_check_reservacion])
len(mismatched_fechas_reservacion)

70559

In [19]:
column_check_llegada = date_columns[4:7]
mismatched_fechas_llegada = date_comparison_check(df[column_check_llegada])
len(mismatched_fechas_llegada)

0

In [20]:
column_check_registro = date_columns[7:10]
mismatched_fechas_registro = date_comparison_check(df[column_check_registro])
len(mismatched_fechas_registro)

0

In [21]:
column_check_salida = date_columns[10:13]
mismatched_fechas_salida = date_comparison_check(df[column_check_salida])
len(mismatched_fechas_salida)

0

In [22]:
column_check_ultima = date_columns[13:]
mismatched_fechas_ultima = date_comparison_check(df[column_check_ultima])
len(mismatched_fechas_ultima)

0

Mismatches en la fecha de reservacion

In [23]:
view(df[column_check_reservacion].loc[mismatched_fechas_reservacion])

,fecha_hoy,h_res_fec,h_res_fec_ok,h_res_fec_okt
102199,20200816,20190816,20190816,20190816
102200,20201022,20191022,20191022,20191022
102201,20201028,20191028,20191028,20191028
102202,20201028,20191028,20191028,20191028
102203,20201028,20191028,20191028,20191028
102204,20200805,20190805,20190805,20190805
102205,20200719,20190719,20190719,20190719
102206,20200719,20190719,20190719,20190719
102207,20200719,20190719,20190719,20190719
102208,20200506,20190506,20190506,20190506


Parece ser que en la columna 'fecha_hoy' esta un año adelantado en algunas fechas se verifica esto. SE ELIMINA

In [24]:
columnas_fechas_eliminar = ['fecha_hoy']

In [25]:
df.drop(columns = columnas_fechas_eliminar, inplace = True)

Ya no hay columnas con datos erroneos, ahora solo nos quedamos con una sola fecha

In [26]:
date_columns = [col for col in date_columns if col not in columnas_fechas_eliminar]

In [27]:
positions_to_keep = [0, 3, 6, 9, 13]
columns_to_keep = [date_columns[i] for i in positions_to_keep]
df.drop(columns = [col for col in date_columns if col not in columns_to_keep], inplace = True)

### Eliminar columnas irrelevantes para el analisis
- Columnas con desviacion estandar igual a 0 (mismo valor en todas las filas)
- Columnas insignificantes para la serie de tiempo y tda

In [28]:
df.describe()

,id_reserva,h_num_per,h_num_adu,h_num_men,cuartos_noche,habitaciones,id_programa,id_paquete,id_empresa,id_pais_origen,cliente_disp,reservacion,ingresos_por_habitacion,moneda_cve
count,145893.000000,145893.000000,145893.000000,145893.000000,145893.000000,145893.000000,145893.000000,145893.000000,145893.0,145893.000000,145893.000000,145893.000000,1.458930e+05,145893.0
mean,100804.750296,1.260725,1.142029,0.043491,1.815234,0.520450,0.999883,1.315587,1.0,156.447993,1.142029,0.516365,4.191225e+03,1.0
std,59181.695722,1.434976,1.244432,0.291197,3.220616,0.510361,0.010794,0.473228,0.0,11.994259,1.244432,0.499734,6.873446e+03,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,-3.091000e+04,1.0
25%,51841.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.0,157.000000,0.000000,0.000000,0.000000e+00,1.0
50%,98559.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.0,157.000000,1.000000,1.000000,0.000000e+00,1.0
75%,153370.000000,2.000000,2.000000,0.000000,3.000000,1.000000,1.000000,2.000000,1.0,157.000000,2.000000,1.000000,7.452000e+03,1.0
max,203001.000000,32.000000,15.000000,15.000000,687.000000,6.000000,1.000000,5.000000,1.0,232.000000,15.000000,1.000000,1.033056e+06,1.0


In [29]:
constant_columns = df.std(numeric_only = True)[lambda x: x == 0].index
df.drop(columns = constant_columns, inplace = True)

Comprobamos que las columnas 'reservacion' y 'aa_reservacion' tienen valores inversos en todas las filas, por ende eliminamos una de ellas

In [30]:
df.id_programa.value_counts()

id_programa
1    145876
0        17
Name: count, dtype: int64

Determinamos que es irrelevante esta columna

In [31]:
columnas_eliminar = ['id_programa']

In [32]:
df.id_paquete.value_counts()

id_paquete
1    100117
2     45635
5        87
3        35
0        17
4         2
Name: count, dtype: int64

No tenemos informacion de esta columna y esta muy dispareja. Determinamos que no procederemos a analizar esta columna

In [33]:
columnas_eliminar += ['id_paquete']

In [34]:
df.h_can_res.value_counts()

h_can_res
00    74512
03    36460
09    17581
06     6738
DI     4665
07     2057
12     1960
08      765
04      579
10      314
02      141
01       63
05       37
         17
11        4
Name: count, dtype: int64

No tenemos informacion de esta columna. Determinamos que no procederemos a analizar esta columna

In [35]:
columnas_eliminar += ['h_can_res']

Estas columnas no presentan relevancia para el analisis.

In [36]:
df.columns

Index(['id_reserva', 'h_res_fec', 'h_num_per', 'h_num_adu', 'h_num_men',
       'cuartos_noche', 'habitaciones', 'id_programa', 'id_paquete',
       'fecha_llegada', 'h_fec_reg', 'fecha_salida', 'id_pais_origen',
       'cliente_disp', 'reservacion', 'h_can_res', 'n_de_confirmacion',
       'estado', 'codigo_postal', 'ingresos_por_habitacion',
       'fecha_confirmacion_pickup', 'agencia', 'canal',
       'estatus_de_la_reservacion', 'segmento_alterno',
       'tipo_habitacion_nombre'],
      dtype='object')

In [37]:
columnas_eliminar += ['n_de_confirmacion', 'codigo_postal', 'id_pais_origen', 'estado']

In [38]:
df.drop(columns = columnas_eliminar, inplace = True)

### Renombrar columnas

In [39]:
df.rename(columns = {
    'h_res_fec': 'fecha_reservacion',
    'h_fec_reg': 'fecha_registro',
    'h_num_per': 'numero_personas',
    'h_num_adu': 'numero_adultos',
    'h_num_men': 'numero_menores',
    'cliente_disp': 'cliente_disponible',
    'cuartos_noche': 'cantidad_noches'
}, inplace = True)

### Revisar por duplicaciones post-edicion y Guardar archivo para los analisis

In [40]:
duplicated_data = df.drop('id_reserva', axis = 1).duplicated()
duplicated_data = duplicated_data[duplicated_data]
# duplicated_data.sum()

In [41]:
df.drop(duplicated_data.index, inplace = True)

In [ ]:
df.to_csv("../Data_Sources/EDA_data.csv", index = False)